In [1]:
import numpy as np
import random
from collections import namedtuple, deque

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
from unityagents import UnityEnvironment
import numpy as np
import torch

In [3]:
from ddpg_agent import DDPGAgent

In [4]:
env = UnityEnvironment(file_name='Reacher_Linux1/Reacher.x86_64')
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=False)[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [5]:
# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]

Number of agents: 1
Size of each action: 4


In [6]:
in_actor = state_size
in_critic = state_size + action_size

hidden_in_actor = 256
hidden_out_actor = 128
hidden_in_critic = 256
hidden_out_critic = 128

out_actor = action_size

In [7]:
from model import DDPGNet
target_actor = DDPGNet(in_actor, hidden_in_actor, hidden_out_actor, out_actor, actor = True)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
target_actor.load_state_dict(torch.load('best_checkpoint500.pth'))
target_actor.to(device)

DDPGNet(
  (fc1): Linear(in_features=33, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=4, bias=True)
)

In [10]:
def actor_act(target_actor, state, eps = 0.0):
    state = torch.from_numpy(state).float().unsqueeze(0).to(device)
    target_actor.eval()
    with torch.no_grad():
        action_values = target_actor(state)
    target_actor.train()
    action_vals_npy = action_values.cpu().data.numpy()
    return np.clip(action_vals_npy, -1, 1)

In [ ]:
# load the weights from file
env_info = env.reset(train_mode = False)[brain_name]
state = env_info.vector_observations[0]
score = 0
while True:
    action = actor_act(target_actor, state, eps = 0.0) #target_actor.act(state, eps)
    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]
    reward = env_info.rewards[0]
    done = env_info.local_done[0]
    #agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
        break
    
print("Score: {}".format(score))

In [12]:
env.close()